In [180]:
using Base
using Dates
using Missings
using ShiftedArrays

using BenchmarkTools
using DataFrames
@using DataFramesMeta
using CSV
using Glob

LoadError: LoadError: [91mUndefVarError: @using not defined[39m
in expression starting at In[180]:8

In [181]:
function find_files(path, pattern, ext)
    if isempty(pattern)
        files=glob("**/*"*ext, path) #Vector of filenames. Glob allows you to use the asterisk.
        println("Number of files: ", length(files))    #Number of files to read.
        return files
    elseif !isempty(pattern)
        files=glob("**/"*pattern*ext, path) #Vector of filenames. Glob allows you to use the asterisk.
        println("Number of files: ", length(files))    #Number of files to read.
        return files
    end
    return []
end

find_files (generic function with 1 method)

In [182]:
parent_dir = "D:/Libraries/Documents/Climatology_Personal/USCRN_USRCRN_SUBHOURLY/"
pattern = "*-MN_Good*"

files = find_files(parent_dir,pattern,".txt")
join_col= ""
header_ = ["WBANNO", "UTC_DATE", "UTC_TIME", "LST_DATE", "LST_TIME", "CRX_VN", "LONGITUDE",
    "LATITUDE", "AIR_TEMPERATURE", "PRECIPITATION", "SOLAR_RADIATION", "SR_FLAG",
    "SURFACE_TEMPERATURE", "ST_TYPE", "ST_FLAG", "RELATIVE_HUMIDITY", "RH_FLAG",
    "SOIL_MOISTURE_5", "SOIL_TEMPERATURE_5", "WETNESS WET_FLAG", "WIND_1_5", "WIND_FLAG"]
types_ = [Int; fill(String, 5); fill(Float64, 5); String; Float64; fill(String,2);
    Float64; String; fill(Float64,3); String; Float64; String]
select_ = ["WBANNO","UTC_DATE","UTC_TIME","LST_DATE","LST_TIME","LONGITUDE","LATITUDE",
    "AIR_TEMPERATURE"]
missingstrings_ = ["","","","","","","","-9999.0","","","","-9999.0","","","","","","-9999.0","","",""]
temp_dfs=Vector{DataFrame}(undef, length(files)) #Create a vector of empty dataframes.
for i in 1:length(files)
    temp_dfs[i]=CSV.read(files[i],DataFrame,ignorerepeated=true,delim=' ',
        header=header_,types=types_, select=select_, missingstrings=missingstrings_,
        silencewarnings=true) #Read each  infileto its own dataframe.
end

df = reduce(vcat, temp_dfs)
describe(df)

#masterdf=join(temp_dfs..., kind = :cross, makeunique=false) #Join the temporary dataframes into one dataframe. 
#masterdf=outerjoin(tempdfs..., on="Column In Common") #Join the temporary dataframes into one dataframe. 


Number of files: 16


,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Union…,Type
1,WBANNO,4994.0,4994,4994.0,4994,,,Int64
2,UTC_DATE,,20060101,,20210302,5540,,String
3,UTC_TIME,,0000,,2355,288,,String
4,LST_DATE,,20051231,,20210302,5541,,String
5,LST_TIME,,0000,,2355,288,,String
6,LONGITUDE,-95.87,-95.87,-95.87,-95.87,,,Float64
7,LATITUDE,48.31,48.31,48.31,48.31,,,Float64
8,AIR_TEMPERATURE,3.61848,-42.0,4.4,36.0,,3810,"Union{Missing, Float64}"


?CSV.read

for (x,y) in enumerate(zip(eltype.(eachcol(df)),names(df)))
    @show x, y
    
end

In [183]:
dropmissing!(df::AbstractDataFrame, "AIR_TEMPERATURE")

date_format = DateFormat("yyyymmdd HHMM")
df.UTC_DATETIME = DateTime.(df.UTC_DATE.*" ".*df.UTC_TIME,date_format)
df.LST_DATETIME = DateTime.(df.LST_DATE.*" ".*df.LST_TIME,date_format)
select!(df, Not([:UTC_DATE, :UTC_TIME, :LST_DATE, :LST_TIME]))

for (x,y) in enumerate(zip(eltype.(eachcol(df)),names(df)))
    @show x, y
    
end

(x, y) = (1, (Int64, "WBANNO"))
(x, y) = (2, (Float64, "LONGITUDE"))
(x, y) = (3, (Float64, "LATITUDE"))
(x, y) = (4, (Float64, "AIR_TEMPERATURE"))
(x, y) = (5, (DateTime, "UTC_DATETIME"))
(x, y) = (6, (DateTime, "LST_DATETIME"))


In [184]:
temp_thresh = -30.
df.THRESH_ORDINAL_RANK = sign.(df.AIR_TEMPERATURE.-temp_thresh)
df.RUN_LENGTH = (df.THRESH_ORDINAL_RANK .- lag(df.THRESH_ORDINAL_RANK)).!= 0
df.RUN_LENGTH[:1] = 0  #We set this to 0 due to the diff() fuction 1 line above, we do not know what occured t=-1 records ago
df.RUN_LENGTH = cumsum(convert(Array{Union{Missing, Int64}}, df.RUN_LENGTH))

1591662-element Array{Union{Missing, Int64},1}:
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    ⋮
 1207
 1207
 1207
 1207
 1207
 1207
 1207
 1207
 1207
 1207
 1207
 1207

In [185]:
gdf = groupby(df,:RUN_LENGTH)
temp_dfs=Vector{DataFrame}(undef, length(gdf)) #Create a vector of empty dataframes.
gdf[1]
for i in 1:length(gdf)  
    temp_dfs[i]= DataFrame(UTC_BEGIN_DATETIME = first(gdf[i].UTC_DATETIME),
        UTC_END_DATETIME = last(gdf[i].UTC_DATETIME),
        UTC_DURATION = last(gdf[i].UTC_DATETIME)- first(gdf[i].LST_DATETIME),
        LST_BEGIN_DATETIME = first(gdf[i].LST_DATETIME),
        LST_END_DATETIME = last(gdf[i].LST_DATETIME),
        THRESH_ORDINAL_RANK = first(gdf[i].THRESH_ORDINAL_RANK),
        MEAN_TEMP = mean(gdf[i].AIR_TEMPERATURE),
        MAX_TEMP = maximum(gdf[i].AIR_TEMPERATURE),
        MIN_TEMP = minimum(gdf[i].AIR_TEMPERATURE)

    )
end

res = reduce(vcat, temp_dfs)
#describe(df)

#consecutive_group = [parent(gdf)[i, groupvars(gdf)] for i in gdf.starts]
#res = DataFrame(MEAN_TEMPERATURE = mean.(consecutive_group.AIR_TEMPERATURE))

,UTC_BEGIN_DATETIME,UTC_END_DATETIME,UTC_DURATION,LST_BEGIN_DATETIME
,DateTime,DateTime,Millise…,DateTime
1,2006-01-01T00:05:00,2006-02-16T07:45:00,4023600000 milliseconds,2005-12-31T18:05:00
2,2006-02-16T07:50:00,2006-02-16T07:55:00,21900000 milliseconds,2006-02-16T01:50:00
3,2006-02-16T08:00:00,2006-02-16T08:00:00,21600000 milliseconds,2006-02-16T02:00:00
4,2006-02-16T08:05:00,2006-02-16T08:35:00,23400000 milliseconds,2006-02-16T02:05:00
5,2006-02-16T08:40:00,2006-02-16T08:50:00,22200000 milliseconds,2006-02-16T02:40:00
6,2006-02-16T08:55:00,2006-02-16T09:00:00,21900000 milliseconds,2006-02-16T02:55:00
7,2006-02-16T09:05:00,2006-02-16T09:15:00,22200000 milliseconds,2006-02-16T03:05:00
8,2006-02-16T09:20:00,2006-02-16T09:50:00,23400000 milliseconds,2006-02-16T03:20:00
9,2006-02-16T09:55:00,2006-02-16T10:10:00,22500000 milliseconds,2006-02-16T03:55:00


In [186]:
#df.RUN_LENGTH
describe(res)

,variable,mean,min,median,max
,Symbol,Union…,Any,Union…,Any
1,UTC_BEGIN_DATETIME,,2006-01-01T00:05:00,,2021-02-17T12:55:00
2,UTC_END_DATETIME,,2006-02-16T07:45:00,,2021-03-02T20:00:00
3,UTC_DURATION,,21600000 milliseconds,,31788900000 milliseconds
4,LST_BEGIN_DATETIME,,2005-12-31T18:05:00,,2021-02-17T06:55:00
5,LST_END_DATETIME,,2006-02-16T01:45:00,,2021-03-02T14:00:00
6,THRESH_ORDINAL_RANK,0.00827815,-1.0,0.0,1.0
7,MEAN_TEMP,-28.6093,-36.4704,-30.0,9.03029
8,MAX_TEMP,-27.1237,-31.2,-30.0,36.0
9,MIN_TEMP,-30.4815,-42.0,-30.0,-28.7


In [187]:
#@where(res, :THRESH_ORDINAL_RANK .== -1)
show(sort!(res[res[:,:THRESH_ORDINAL_RANK] .== -1, :], [:UTC_DURATION, :MEAN_TEMP]), true)

440×9 DataFrame
│ Row │ UTC_BEGIN_DATETIME  │ UTC_END_DATETIME    │ UTC_DURATION           │
│     │ DateTime            │ DateTime            │ Millisecond            │
├─────┼─────────────────────┼─────────────────────┼────────────────────────┤
│ 1   │ 2018-01-13T07:00:00 │ 2018-01-13T07:00:00 │ 21600000 milliseconds  │
│ 2   │ 2009-02-03T12:20:00 │ 2009-02-03T12:20:00 │ 21600000 milliseconds  │
│ 3   │ 2017-01-05T02:05:00 │ 2017-01-05T02:05:00 │ 21600000 milliseconds  │
│ 4   │ 2019-02-09T02:25:00 │ 2019-02-09T02:25:00 │ 21600000 milliseconds  │
│ 5   │ 2007-12-14T10:55:00 │ 2007-12-14T10:55:00 │ 21600000 milliseconds  │
│ 6   │ 2020-01-11T03:40:00 │ 2020-01-11T03:40:00 │ 21600000 milliseconds  │
│ 7   │ 2011-01-03T03:55:00 │ 2011-01-03T03:55:00 │ 21600000 milliseconds  │
│ 8   │ 2016-02-13T06:35:00 │ 2016-02-13T06:35:00 │ 21600000 milliseconds  │
│ 9   │ 2017-01-05T04:25:00 │ 2017-01-05T04:25:00 │ 21600000 milliseconds  │
│ 10  │ 2017-12-27T03:45:00 │ 2017-12-27T03:45:00 │ 21600000